In [2]:
import urllib
from urllib.request import urlopen
import time
import json
import pickle
import time
import pandas as pd
from bs4 import BeautifulSoup
import re
import dateparser
from datetime import datetime

In [3]:
page=1
valid_page = True
base_url="http://www.fixmystreet.com/reports"
borough="Lambeth"

pins = []
reports = []
max_pages=30
max_retries=10
date_match_r=re.compile("^([ ]+)?[0-2][0-9].*")

while valid_page and page <= max_pages:
    retries = max_retries
    while retries > 0 :
        try:
            url_params="?ajax=1&filter_category=Flytipping,Rubbish%20(refuse%20and%20recycling),Street%20cleaning&sort=created-desc&p={}&bbox=-0.27885783939054,51.365864618287,0.049358713343836,51.554746382133".format(page)
            req_url="{}/{}/{}".format(base_url,borough,url_params)
            hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'}
            req = urllib.request.Request(req_url,headers=hdr)
            site_data = urlopen(req)
            data = json.load(site_data)
            if not data['pins']:
                break
            pins += data['pins']
            soup = BeautifulSoup(data['reports_list'], "html.parser")
            for entry in soup.find_all('li'):
                fixed = True if entry.select_one("span[class=item-list__item__state]") else False
                date_str = list(filter(date_match_r.match, entry.small.get_text().strip().splitlines()))
                date_str = [x.strip() for x in date_str]
                report_date = dateparser.parse(date_str[0])
                reports.append([int(entry["data-report-id"]),report_date,dateparser.parse(entry["data-lastupdate"]),fixed])
            time.sleep(10)
        except Exception as e:
            print("error retrieving : {}".format(req_url))
            print(e)
            retries = retries - 1
            retry_count = max_retries-retries
            time.sleep(5*retry_count)
            continue
        else:
            break
    print("page: {}, retries: {}".format(page,max_retries-retries))
    page += 1
    if not data['pins'] or retries is 0:
        break

page: 1, retries: 0
page: 2, retries: 0
page: 3, retries: 0
page: 4, retries: 0
page: 5, retries: 0
page: 6, retries: 0
page: 7, retries: 0
page: 8, retries: 0
page: 9, retries: 0
page: 10, retries: 0
page: 11, retries: 0
page: 12, retries: 0
page: 13, retries: 0
page: 14, retries: 0
page: 15, retries: 0
page: 16, retries: 0
page: 17, retries: 0
page: 18, retries: 0
page: 19, retries: 0
page: 20, retries: 0
page: 21, retries: 0
page: 22, retries: 0
page: 23, retries: 0


In [4]:
columns_pins=['lat','lon','status','id','summary']
colums_reports=['id','date','last_update','fixed']
df_pins = pd.DataFrame(pins,columns=columns_pins)
df_reports = pd.DataFrame(reports,columns=colums_reports)

In [5]:
df_reports.head()

,id,date,last_update,fixed
0,1159649,2017-12-11 12:10:00,2017-12-11 12:15:12,False
1,1159029,2017-12-09 23:11:00,2017-12-09 23:15:04,False
2,1159028,2017-12-09 23:09:00,2017-12-09 23:10:05,False
3,1159027,2017-12-09 23:08:00,2017-12-09 23:10:04,False
4,1158967,2017-12-09 19:14:00,2017-12-09 19:15:06,False


In [6]:
df = pd.merge(df_pins,df_reports)
df.head()

,lat,lon,status,id,summary,date,last_update,fixed
0,51.419381,-0.128119,yellow,1159649,Pavement wide puddle outside The Bill pub. Ple...,2017-12-11 12:10:00,2017-12-11 12:15:12,False
1,51.420878,-0.129265,yellow,1159029,Fridge and furniture flytipped next to garage,2017-12-09 23:11:00,2017-12-09 23:15:04,False
2,51.423701,-0.130316,yellow,1159028,Furniture flytipped by st Reatham Antiques,2017-12-09 23:09:00,2017-12-09 23:10:05,False
3,51.422149,-0.129984,yellow,1159027,Rubbish building up in pavement,2017-12-09 23:08:00,2017-12-09 23:10:04,False
4,51.4169,-0.126273,yellow,1158967,Rubbish flytipped on pavement,2017-12-09 19:14:00,2017-12-09 19:15:06,False


In [7]:
df['lat'] = df['lat'].astype('float64') 
df['lon'] = df['lon'].astype('float64')

df.to_pickle('{}-report-data.pickle'.format(time.strftime("%Y%m%d%H%M%S")))